## **Preparación y limpieza de datos**

#### 📘 **Importación de librerías** 
Importamos las librerías que necesitaremos en nuestra **limpieza** para los análisis posteriores

In [16]:
import numpy as np
import pandas as pd
from statistics import mode

#### 🛜 **Importación del dataset Telco**

Este dataset proviene de la cartera de una empresa de telecomunicaciones que da servicio al Valle y al Tolima. 


In [17]:
## Cargar datos 
df = pd.read_csv("./data/telco-dataset.csv")

## respaldo - backup 
df_original = df.copy()

df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 165644 entries, 0 to 165643
Data columns (total 21 columns):
 #   Column              Non-Null Count   Dtype  
---  ------              --------------   -----  
 0   CLIENTE_ID_UI       165644 non-null  object 
 1   TARIFA_PS           165644 non-null  object 
 2   ESTADO_CLIENTE      165644 non-null  object 
 3   ESTADO_SUMINISTRO   165644 non-null  object 
 4   ESTADO_FACTURACION  165644 non-null  object 
 5   DEUDA               165644 non-null  int64  
 6   ANTIGUEDAD_SALDO    165644 non-null  int64  
 7   GRUPO_CU            165644 non-null  object 
 8   CICLO               165644 non-null  int64  
 9   ZONA                165639 non-null  float64
 10  CLASE_SERVICIO      165644 non-null  object 
 11  ESTRATO             165644 non-null  int64  
 12  DEPTO               165644 non-null  object 
 13  MUNICIPIO           165644 non-null  object 
 14  BARRIO              165644 non-null  object 
 15  TIPO_SERVICIO       165644 non-nul

In [18]:
df.head(3)

,CLIENTE_ID_UI,TARIFA_PS,ESTADO_CLIENTE,ESTADO_SUMINISTRO,ESTADO_FACTURACION,DEUDA,ANTIGUEDAD_SALDO,GRUPO_CU,CICLO,ZONA,...,ESTRATO,DEPTO,MUNICIPIO,BARRIO,TIPO_SERVICIO,TOTAL_FACTURADO,FECHA_FACTURACION,FECHA_VENCIMIENTO,FECHA_ULT_PAGO,VALOR_ULT_PAGO
0,d91b230b-8619-53e2-8772-52cc8ce140be,189-PLAN 100 MEGAS 2025,30-Inactivo,0-Con suministro,0-Sin Proceso,282100,4,1-TOLIMA,6,1.0,...,2,73-Tolima,73001-IBAGUÉ,1883-URB. ARKAMBUCO I,T-Telecomunicaciones,282100.0,21/04/2025,02/05/2025,NaN,NaN
1,e076edda-bf70-5105-a9a9-118d7eecd0c4,189-PLAN 100 MEGAS 2025,0-Activo,0-Con suministro,2-Calculo de Consumo,486560,7,2-VALLE,1,2.0,...,1,76-Valle del Cauca,76364-JAMUNDÍ,9497-EDEN DEL PARQUE,T-Telecomunicaciones,486560.0,08/07/2025,18/07/2025,NaN,NaN
2,6a8ddfbb-fd47-5320-a6bd-a3b4ab26e9d5,112-PLAN 200 MEGAS 2024,30-Inactivo,0-Con suministro,0-Sin Proceso,0,0,1-TOLIMA,2,1.0,...,3,73-Tolima,73001-IBAGUÉ,1675-LA FRANCIA,T-Telecomunicaciones,NaN,NaN,NaN,NaN,NaN


---
#### **🔎 Revisión de datos perdidos o nulos**
   

In [19]:
df.isnull().sum()

CLIENTE_ID_UI             0
TARIFA_PS                 0
ESTADO_CLIENTE            0
ESTADO_SUMINISTRO         0
ESTADO_FACTURACION        0
DEUDA                     0
ANTIGUEDAD_SALDO          0
GRUPO_CU                  0
CICLO                     0
ZONA                      5
CLASE_SERVICIO            0
ESTRATO                   0
DEPTO                     0
MUNICIPIO                 0
BARRIO                    0
TIPO_SERVICIO             0
TOTAL_FACTURADO        3780
FECHA_FACTURACION      3780
FECHA_VENCIMIENTO      3780
FECHA_ULT_PAGO        20679
VALOR_ULT_PAGO        20679
dtype: int64

---
**ZONA:** `ZONA` es una feature que clasifica las ubicaciones en el municipio, esta feature es util para la entrega de facturas. Ya que solo son 5 nulos, se imputa con la moda debido a que es una variable categorica. 

In [20]:
## Vemos el tipo de dato de zona y sus diferentes valores en el df
df['ZONA'].value_counts()

ZONA
2.0    101970
1.0     63667
3.0         2
Name: count, dtype: int64

In [21]:
## Imputamos con la moda estos 5 registros
df['ZONA'] = df['ZONA'].fillna(df['ZONA'].mode()[0])
df['ZONA'].value_counts() ##Se evidencia el aumento de 5 unidades en la moda 2

ZONA
2.0    101975
1.0     63667
3.0         2
Name: count, dtype: int64

---
**VALOR_ULT_PAGO**: Los nulos en este feature indican aquellos clientes que nunca han pagado, por esa razón imputaremos con 0 estos valores 

**VALOR_FACTURADO**: Los nulos en este caso se llenarán con 0 pero se agregará una columna para indicar si se era nula o no. Esto con el fin de no perder la trazabilidad de aquellos datos cuyo valor facturado no existia. 

In [22]:
## valor ult pago
df['VALOR_ULT_PAGO'] = df['VALOR_ULT_PAGO'].fillna(0)

## total_facturado
df['TOTAL_FACTURADO_NULL'] = df['TOTAL_FACTURADO'].isna().astype(int)
df['TOTAL_FACTURADO'] = df['TOTAL_FACTURADO'].fillna(0)

---
**FECHA_FACTURACION** -
**FECHA_VENCIMIENTO** - 
**FECHA_ULT_PAGO** 

Para las variables de fecha se hará lo mismo, se llenarán aquellos datos que indiquen que no tienen factura al igual que se crearán columnas dummies para indicar si tienen o no fecha

In [23]:
## Fechas 
df['FACTURA_NULL'] = df['FECHA_FACTURACION'].notna().astype(int)
df['FACTURA_VENCIMIENTO_NULL'] = df['FECHA_VENCIMIENTO'].notna().astype(int)
df['PAGO_NULL'] = df['FECHA_ULT_PAGO'].notna().astype(int)

df['FECHA_FACTURACION'] = df['FECHA_FACTURACION'].fillna("SIN_FACTURA")
df['FECHA_VENCIMIENTO'] = df['FECHA_VENCIMIENTO'].fillna("SIN_FECHA")
df['FECHA_ULT_PAGO'] = df['FECHA_ULT_PAGO'].fillna("SIN_PAGO")

In [24]:
df.isnull().sum()

CLIENTE_ID_UI               0
TARIFA_PS                   0
ESTADO_CLIENTE              0
ESTADO_SUMINISTRO           0
ESTADO_FACTURACION          0
DEUDA                       0
ANTIGUEDAD_SALDO            0
GRUPO_CU                    0
CICLO                       0
ZONA                        0
CLASE_SERVICIO              0
ESTRATO                     0
DEPTO                       0
MUNICIPIO                   0
BARRIO                      0
TIPO_SERVICIO               0
TOTAL_FACTURADO             0
FECHA_FACTURACION           0
FECHA_VENCIMIENTO           0
FECHA_ULT_PAGO              0
VALOR_ULT_PAGO              0
TOTAL_FACTURADO_NULL        0
FACTURA_NULL                0
FACTURA_VENCIMIENTO_NULL    0
PAGO_NULL                   0
dtype: int64

#### **🔎 Datos duplicados, redudantes o poco relevantes**
Se buscan variables que no sean relevantes o que estén repetidas en el dataset. 

In [25]:
## Filas duplicadas
df.duplicated().sum()

np.int64(0)

In [26]:
## Tipos de datos en columnas
df.nunique()

CLIENTE_ID_UI               165644
TARIFA_PS                       65
ESTADO_CLIENTE                   4
ESTADO_SUMINISTRO               10
ESTADO_FACTURACION               3
DEUDA                        21793
ANTIGUEDAD_SALDO                34
GRUPO_CU                         3
CICLO                            9
ZONA                             3
CLASE_SERVICIO                   6
ESTRATO                          7
DEPTO                            4
MUNICIPIO                       42
BARRIO                        2262
TIPO_SERVICIO                    1
TOTAL_FACTURADO              22955
FECHA_FACTURACION             1071
FECHA_VENCIMIENTO              835
FECHA_ULT_PAGO                1355
VALOR_ULT_PAGO               15554
TOTAL_FACTURADO_NULL             2
FACTURA_NULL                     2
FACTURA_VENCIMIENTO_NULL         2
PAGO_NULL                        2
dtype: int64

**TIPO_SERVICIO**: Se identifica feature TIPO_SERVICIO que solo contiene 1 tipo de dato que es T - Telecomunicaciones, se procede a eliminar ya que no es relevante para el negocio (solo se tiene un servicio que es telecomunicaciones) 

**GRUPO_CU**: Esta feature indica a que grupo del departamento pertenece el cliente, sin embargo, ya existe una feature `DEPTO` por esa razón se procede a dejar solo DEPTO

In [27]:
print("Valores únicos en DEPTO:", df["DEPTO"].unique())
print("Valores únicos en GRUPO_CU:", df["GRUPO_CU"].unique())

Valores únicos en DEPTO: ['73-Tolima' '76-Valle del Cauca' '19-Cauca' '25-Cundinamarca']
Valores únicos en GRUPO_CU: ['1-TOLIMA' '2-VALLE' '3-CUNDINAMARCA']


**CLIENTE_ID_UI**: Si bien el dataset ya vino con ciertos filtros desde el negocio por temas de privacidad y sensibilidad de la empresa, aun se tiene `CLIENTE_ID_UI` que trae puede traer información sensible 1 a 1 del cliente (Data Leakage)

In [28]:
FEATURES_TO_DELETE = ['TIPO_SERVICIO', 'GRUPO_CU', 'CLIENTE_ID_UI']
df = df.drop(columns=FEATURES_TO_DELETE)

---
#### **Export del df** 


In [29]:
route = './data/clientes_limpios.csv'
df.to_csv(route, index=False, sep=';', encoding='utf-8')